# Chapter 3: vectorize contextual information

In this chapter you will take a document, split it into chunks and embed the chunks (transform their meaning into vectors). You will then save the vectorized chunks into a csv file.

## Download a document

In [ ]:
pip install requests

In [ ]:
import requests

document_link = ...

document = requests.get(document_link)

In [ ]:
print(document)

## Parse it into chunks

## Vectorize the chunks

## Save the vector-chunk pairs